In [1]:
import os
import sys
import pandas as pd

In [2]:
partition = input("Inserisci la partizione che vuoi utilizzare")
domain = "time"
#domain = "fourier_fourier"
#domain = "fourier_t"
#domain = "wavelet_t"
#domain = "wavelet_wv"

print("Hai inserito la partizione: ", partition)
#partition = os.path.join("../partitioning/npy", partition)

foldername = input("Inserisci il nome della cartella dove salvare i risultati")
print("Hai inserito la cartella: ", foldername)
foldername = os.path.join(domain, "training_xgboost", foldername)
print(foldername)

Hai inserito la partizione:  test_t_t_5
Hai inserito la cartella:  test_5
time/training_xgboost/test_5


In [3]:
import tensorflow as tf
print("GPU disponibile:", tf.config.list_physical_devices('GPU'))

2025-04-02 12:13:37.855903: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-X.Y/lib64
2025-04-02 12:13:37.855926: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


GPU disponibile: []


2025-04-02 12:13:45.938913: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-04-02 12:13:45.938963: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (paolo-Katana-15-B13VFK): /proc/driver/nvidia/version does not exist


In [4]:
sys.path.append("../models")

In [5]:
from xgboost_model import XGBoostModel

In [6]:
def load_partitions(root_path):
    file_names = ["X_train.csv", "X_test.csv", "X_val.csv", "y_train.csv", "y_test.csv", "y_val.csv"]
    datasets = [pd.read_csv(os.path.join(root_path, file)).values for file in file_names]
    return tuple(datasets)

In [7]:
# load data
partition = os.path.join("../partitioning/partitions", partition)
X_train, X_test, X_val, y_train, y_test, y_val = load_partitions(partition)

# show partitions dimention
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

X_train shape: (16289, 10), y_train shape: (16289, 1)
X_test shape: (3491, 10), y_test shape: (3491, 1)
X_val shape: (3489, 10), y_val shape: (3489, 1)


In [8]:
# initialize model
xgboost_model = XGBoostModel(foldername, n_estimators=100, output_units=y_train.shape[1])

In [9]:
# training
xgboost_model.train(X_train, y_train, X_val, y_val)

[0]	train-rmse:0.12714	train-mae:0.09811	eval-rmse:0.12863	eval-mae:0.09907
[1]	train-rmse:0.12052	train-mae:0.09285	eval-rmse:0.12216	eval-mae:0.09396
[2]	train-rmse:0.11485	train-mae:0.08839	eval-rmse:0.11653	eval-mae:0.08954
[3]	train-rmse:0.11008	train-mae:0.08462	eval-rmse:0.11192	eval-mae:0.08591
[4]	train-rmse:0.10599	train-mae:0.08139	eval-rmse:0.10795	eval-mae:0.08280
[5]	train-rmse:0.10239	train-mae:0.07856	eval-rmse:0.10461	eval-mae:0.08008
[6]	train-rmse:0.09947	train-mae:0.07626	eval-rmse:0.10188	eval-mae:0.07788
[7]	train-rmse:0.09703	train-mae:0.07437	eval-rmse:0.09957	eval-mae:0.07609
[8]	train-rmse:0.09494	train-mae:0.07273	eval-rmse:0.09763	eval-mae:0.07451
[9]	train-rmse:0.09303	train-mae:0.07124	eval-rmse:0.09600	eval-mae:0.07317
[10]	train-rmse:0.09138	train-mae:0.06995	eval-rmse:0.09452	eval-mae:0.07193
[11]	train-rmse:0.09001	train-mae:0.06889	eval-rmse:0.09340	eval-mae:0.07100
[12]	train-rmse:0.08960	train-mae:0.06858	eval-rmse:0.09319	eval-mae:0.07081
[13]	trai

In [10]:
xgboost_model.plot_training_metrics()

Nessuna history trovata. Assicurati di aver eseguito il training con set di validazione.


In [11]:
xgboost_model.evaluate(X_test, y_test)

Metriche dettagliate salvate in: time/training_xgboost/test_5/evaluation_metrics_detailed.csv
Metriche medie salvate in: time/training_xgboost/test_5/evaluation_metrics_mean.csv


{'detailed': {'Output': ['Unico'],
  'MSE': [0.007509584921857172],
  'RMSE': [0.08665786128134696],
  'MAE': [0.0655883446087623],
  'R2': [0.5617952902483216]},
 'mean': {'Output': ['Unico'],
  'MSE': [0.007509584921857172],
  'RMSE': [0.08665786128134696],
  'MAE': [0.0655883446087623],
  'R2': [0.5617952902483216]}}

In [12]:
xgboost_model.save_model()

Modello salvato in time/training_xgboost/test_5/xgboost_model.joblib


In [13]:
y_pred = xgboost_model.predict(X_test)

In [14]:
xgboost_model.compare_predictions(y_test, y_pred)

Confronto predizioni salvato in time/training_xgboost/test_5/predictions_comparison.csv


,Valore Reale,Predizione
0,0.227369,0.107987
1,0.174371,0.269550
2,-0.107064,0.006491
3,0.096252,0.061423
4,0.120111,0.089219
...,...,...
3486,0.121480,0.042930
3487,-0.102473,-0.094178
3488,-0.066549,-0.119441
3489,0.111682,0.065441


In [15]:
xgboost_model.save_training_history_txt()

Nessuna history disponibile da salvare.


In [16]:
xgboost_model.save_parameters()

Parametri salvati in time/training_xgboost/test_5/model_parameters.txt
